In [1]:
import os
import json
from openai import OpenAI
import instructor
from pydantic import BaseModel, Field
from typing import List
import numpy as np
from utils import get_structured_feedback, remove_duplicates, Evaluation, CriteraFeedback

In [2]:
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

instructor_client = instructor.from_openai(openai_client)

In [3]:
# read messages from user1-messages.json
with open("user1-messages.json", "r") as f:
	chat_history = json.load(f)

### Storing all evaluations from a conversation

In [4]:
feedbacks = get_structured_feedback(instructor_client, chat_history)

In [5]:
feedbacks

[Evaluation(introduction=CriteraFeedback(strengths=['Effectively framing the argument in the introduction and conclusion'], weaknesses=[], suggestions=[]), structure=CriteraFeedback(strengths=['Clear and logical organization of the essay'], weaknesses=['Consider developing the counterargument section further', 'Include transition sentences between paragraphs for better flow'], suggestions=['Develop the details or statistics in the counterargument section', 'Add transition sentences between paragraphs']), argumentation=CriteraFeedback(strengths=['Use of examples to solidify the argument'], weaknesses=[], suggestions=['Practice using a variety of sentence structures to convey ideas more clearly']), evidence=CriteraFeedback(strengths=[], weaknesses=[], suggestions=[]), conclusion=CriteraFeedback(strengths=[], weaknesses=[], suggestions=[])),
 Evaluation(introduction=CriteraFeedback(strengths=['Effectively framing the argument in the introduction and conclusion'], weaknesses=[], suggestion

### Combining evaluations from a chat session to remove duplicates

In [6]:
def aggregate_feedback(feedbacks: List[Evaluation]):
    # Initialize lists for each feedback category
    aggregated = {
        "introduction": {"strengths": [], "weaknesses": [], "suggestions": []},
        "structure": {"strengths": [], "weaknesses": [], "suggestions": []},
        "argumentation": {"strengths": [], "weaknesses": [], "suggestions": []},
        "evidence": {"strengths": [], "weaknesses": [], "suggestions": []},
        "conclusion": {"strengths": [], "weaknesses": [], "suggestions": []}
    }
    
    # Aggregate all feedbacks
    for feedback in feedbacks:
        for key in aggregated:
            critera_feedback = getattr(feedback, key)
            aggregated[key]['strengths'].extend(critera_feedback.strengths)
            aggregated[key]['weaknesses'].extend(critera_feedback.weaknesses)
            aggregated[key]['suggestions'].extend(critera_feedback.suggestions)
    
    # Deduplicate lists
    for key in aggregated:
        for subkey in aggregated[key]:
            aggregated[key][subkey] = remove_duplicates(openai_client, aggregated[key][subkey])

    # Construct a new Evaluation object
    return Evaluation(
        introduction=CriteraFeedback(**aggregated['introduction']),
        structure=CriteraFeedback(**aggregated['structure']),
        argumentation=CriteraFeedback(**aggregated['argumentation']),
        evidence=CriteraFeedback(**aggregated['evidence']),
        conclusion=CriteraFeedback(**aggregated['conclusion'])
    )

In [7]:
aggregated_feedback = aggregate_feedback(feedbacks)

/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/magicschool-backend-VB4VKBHD-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/13point5/

### Semantic deduplication doesn't seem to work with a dynamic threshold

In [8]:
for criteria, feedback in aggregated_feedback:
	print(f"{criteria}:")
	print(f"Strengths: {feedback.strengths}")
	print(f"Weaknesses: {feedback.weaknesses}")
	print(f"Suggestions: {feedback.suggestions}")
	print("\n")

introduction:
Strengths: ['Effectively framing the argument in the introduction and conclusion', 'Clear framing of the argument']
Weaknesses: []
Suggestions: ['Consider developing the counterargument section further']


structure:
Strengths: ['Clear and logical organization of the essay', 'Clear and logical organization']
Weaknesses: ['Consider developing the counterargument section further', 'Include transition sentences between paragraphs for better flow']
Suggestions: ['Develop the details or statistics in the counterargument section', 'Add transition sentences between paragraphs']


argumentation:
Strengths: ['Use of examples to solidify the argument', 'Thoughtful organization and addressing a counterargument']
Weaknesses: ['Counterargument section could be developed further']
Suggestions: ['Practice using a variety of sentence structures to convey ideas more clearly']


evidence:
Strengths: ['Use of examples like protecting natural habitats and global warming impact', 'Use of exam